# Notebook Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import sklearn as sk

In [4]:
df = pd.read_csv('../data/clean-data/cleaned_nys_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233144 entries, 0 to 233143
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   action_taken                       233144 non-null  int64  
 1   derived_race                       233144 non-null  object 
 2   derived_ethnicity                  233144 non-null  object 
 3   applicant_sex                      233144 non-null  int64  
 4   applicant_age                      233144 non-null  object 
 5   income                             226538 non-null  float64
 6   debt_to_income_ratio               100684 non-null  float64
 7   applicant_credit_score_type        233144 non-null  int64  
 8   loan_amount                        233144 non-null  float64
 9   loan_to_value_ratio                224447 non-null  float64
 10  interest_rate                      176143 non-null  float64
 11  rate_spread                        1519

/var/folders/bk/9ksdjj7n47qdwk9fmh_w5rbc0000gn/T/ipykernel_30308/554199657.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/clean-data/cleaned_nys_data.csv')


# Impact Analysis Using HMDA Loan Approval Data, proxy outcome
regression models (MLR, logistic regression, ML algorithms?) – Victor, Zheka
Keeping predictors similar but testing across different outcome variables (loan amount, interest rate, application outcome)
Switching zip-code with race and testing other outcomes (?)

Model 1: logistic regression that predicts loan outcome
Model 2: loan amount (MLR)
Model 3: interest rate (MLR), rate_spread

In [5]:
df_slice = df.loc[(df['derived_race'] == "Black or African American") | (df['derived_race'] == "White")]
df_race = df_slice.copy()
df_race['binary_race'] = 0
df_race.loc[df_race['derived_race'] == "White", 'binary_race'] = 1

# # transform derived ethnicity to binary column for Hispanic or not
df_race['binary_ethnicity'] = 0
df_race.loc[df_race['derived_ethnicity'] == "Not Hispanic or Latino", 'binary_ethnicity'] = 1

# # create column with 0 for denied applications/preapproval requests and 1 for accepted applications
df_race['binary_denied'] = 0
df_race.loc[(df_race['action_taken'] == 3) | (df_race['action_taken'] == 7), 'binary_denied'] = 1

# # create correlation matrix
df_corr = df_race[['applicant_sex','binary_race','binary_ethnicity','binary_denied']]
df_corr.corr()

,applicant_sex,binary_race,binary_ethnicity,binary_denied
applicant_sex,1.000000,-0.089703,-0.000775,0.021091
binary_race,-0.089703,1.000000,-0.000189,-0.124504
binary_ethnicity,-0.000775,-0.000189,1.000000,-0.063217
binary_denied,0.021091,-0.124504,-0.063217,1.000000


In [7]:
df.columns

Index(['action_taken', 'derived_race', 'derived_ethnicity', 'applicant_sex',
       'applicant_age', 'income', 'debt_to_income_ratio',
       'applicant_credit_score_type', 'loan_amount', 'loan_to_value_ratio',
       'interest_rate', 'rate_spread', 'loan_type', 'loan_purpose',
       'lien_status', 'property_value', 'occupancy_type',
       'tract_minority_population_percent', 'aus-1', 'denial_reason-1'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233144 entries, 0 to 233143
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   action_taken                       233144 non-null  int64  
 1   derived_race                       233144 non-null  object 
 2   derived_ethnicity                  233144 non-null  object 
 3   applicant_sex                      233144 non-null  int64  
 4   applicant_age                      233144 non-null  object 
 5   income                             226538 non-null  float64
 6   debt_to_income_ratio               100684 non-null  float64
 7   applicant_credit_score_type        233144 non-null  int64  
 8   loan_amount                        233144 non-null  float64
 9   loan_to_value_ratio                224447 non-null  float64
 10  interest_rate                      176143 non-null  float64
 11  rate_spread                        1519

In [12]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66615 entries, 0 to 233143
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   action_taken                       66615 non-null  int64  
 1   derived_race                       66615 non-null  object 
 2   derived_ethnicity                  66615 non-null  object 
 3   applicant_sex                      66615 non-null  int64  
 4   applicant_age                      66615 non-null  object 
 5   income                             66615 non-null  float64
 6   debt_to_income_ratio               66615 non-null  float64
 7   applicant_credit_score_type        66615 non-null  int64  
 8   loan_amount                        66615 non-null  float64
 9   loan_to_value_ratio                66615 non-null  float64
 10  interest_rate                      66615 non-null  float64
 11  rate_spread                        66615 non-null  object 

In [ ]:
age_map = {'<25': 1, '25-34': 2, '35-54' : 3, '45-54' : 4, '55-64' : 5, '65-74' : 6, '>74' : 7,'8888' : 8}
df_race['age_categorical'] = df_race['applicant_age'].map(age_map)
# 

In [39]:
import statsmodels.api as sm
df_race = df_race.dropna()
# df_race = df_race.dropna(subset=['interest_rate', 'tract_minority_population_percent', 'applicant_age',
#                                  'debt_to_income_ratio', 'income', "property_value", "loan_to_value_ratio", "loan_amount")

# X = df_race[[ 'binary_race','applicant_sex',
#        'applicant_age', 'income', 'debt_to_income_ratio',
#        'applicant_credit_score_type', 'loan_to_value_ratio', 'loan_type', 'loan_purpose',
#        'lien_status', 'property_value', 'occupancy_type',
#        'tract_minority_population_percent', 'aus-1']]

X = df_race[['tract_minority_population_percent', "binary_race", "debt_to_income_ratio", 'income', "property_value", 'applicant_sex',
             'age_categorical', 'applicant_credit_score_type', 'loan_to_value_ratio', 'loan_type', 'loan_purpose', 'lien_status', 
             'occupancy_type', 'aus-1']]
y = df_race['loan_amount']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            loan_amount   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                 1.381e+04
Date:                Sat, 05 Apr 2025   Prob (F-statistic):               0.00
Time:                        13:56:21   Log-Likelihood:            -6.7285e+05
No. Observations:               50031   AIC:                         1.346e+06
Df Residuals:                   50016   BIC:                         1.346e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

# Recreation of model 
Train a model based on outcome (denial vs acceptance) using similar predictors
Look for bias in the outcomes of the model -> conclusion about biased data creating biased models in future 

In [129]:
from sklearn.model_selection import train_test_split

# select columns that were explicitly approved or denied
filtered_df = df.loc[(df["action_taken"] == 1) | (df["action_taken"] == 2) | (df["action_taken"] == 7)]

# remove na values
filtered_df = filtered_df.dropna(subset=["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"])

# create binary accepted column
filtered_df["binary_accepted"] = True
filtered_df.loc[filtered_df["action_taken"] == 7, 'binary_accepted'] = False
counts = filtered_df['binary_accepted'].value_counts()

# split data

X = filtered_df[["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"]]
y = filtered_df["binary_accepted"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [131]:
from sklearn.linear_model import LinearRegression
model = sk.linear_model.LinearRegression().fit(X,y)

In [137]:
# report accuracy on test set
print(model.score(X_test, y_test)) # Output: 1.0

0.0002840976401518036
